In [25]:
pip install torch torchvision matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
import torch
import torch.nn as nn
import torch.optim as opt
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [27]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5))])
train_dataset = torchvision.datasets.MNIST(root="./data",train=True,transform=transform,download=True)
test_dataset = torchvision.datasets.MNIST(root="./data",train=False,transform=transform,download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=64,shuffle=False)

In [28]:
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super(LinearRegressionModel,self).__init__()
    self.linear = nn.Linear(28*28,10)

  def forward(self,x):
    x = x.view(-1,28*28)
    return self.linear(x)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = LinearRegressionModel().to(device)

Using device: cuda


In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = opt.SGD(model.parameters(),lr = 0.01)

In [31]:
num_epochs = 5

for epoch in range(num_epochs):
  total_loss = 0
  for images, labels in train_loader:
    images,labels = images.to(device),labels.to(device)

    outputs = model(images)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss += loss.item()

  print(f"Epoch [{epoch+1}/{num_epochs}],Loss: {total_loss/len(train_loader):.4f}")

print("Training Complete. Now. let's see how bad this thing performs.")

Epoch [1/5],Loss: 0.6131
Epoch [2/5],Loss: 0.3865
Epoch [3/5],Loss: 0.3523
Epoch [4/5],Loss: 0.3347
Epoch [5/5],Loss: 0.3238
Training Complete. Now. let's see how bad this thing performs.


In [32]:
correct = 0
total = 0

with torch.no_grad():
  for  images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {100*correct/total}%")
# Previous Accuracy on the test set: 91.42% when num_epochs = 5
# Accuracy on the test set: 92.12% when num_epochs = 50

Accuracy on the test set: 91.39%
